# 아래의 블럭은 시작 전에 꼭 실행

In [1]:
import time
import logging
from binance.lib.utils import config_logging
from binance.websocket.um_futures.websocket_client import UMFuturesWebsocketClient
from binance.um_futures import UMFutures
from binance.error import ClientError

from trade_rules.trade_rules import GetData
from cert import binanceKey
from cert import test_binanceKey


# import API KEY and SECRET KEY
## realnet (base_url is realnet default)
API_KEY = binanceKey.API_KEY
SECRET_KEY = binanceKey.SECRET_KEY
## testnet
key = test_binanceKey.API_KEY
secret = test_binanceKey.SECRET_KEY
## testnet base_urls (for test trading, stream_url = futures_websocket_testnet and base_url = futures_testnet)
futures_testnet = test_binanceKey.futures_testnet
futures_websocket_testnet = test_binanceKey.futures_websocket_testnet
# END


# defining futures client
um_futures_client = UMFutures(key=key, secret=secret, base_url=futures_testnet) # testnet
my_client = UMFuturesWebsocketClient(stream_url=futures_websocket_testnet) # testnet
# END

## Order가 각각 어떻게 만들어지는지 확인

In [ ]:
'''
# sending new order에 필요한 parameter 목록

- symbol*
- side*
- positionSide
- type*
- timeInForce
- quantity
- reduceOnly
- price
- newClientOrderId
- stopPrice
- closePosition
- activationPrice
- callbackRate
- workingType
- priceProtect
- newOrderRespType
- recvWindow
- timestamp*


## 어떤 형태의 order든 필수로 필요한 파라미터

symbol, side, type, timestamp
symbol = btcusdt 쓰면 되고.
side, type이 문제임


## type에 들어가는 파라미터의 값에 따라 additional한 mandatory parameter 목록

LIMIT : timeInForce, quantity, price
MARKET : quantity
STOP/TAKE_PROFIT : quantity, price, stopPrice, (timeInForce)
STOP_MARKET/TAKE_PROFIT_MARKET : stopPrice
TRAILING_STOP_MARKET : callbackRate


### LIMIT


'''

In [2]:
## 현재 asset 확인하는 함수 만들기
def get_account(UMclient):
    try:
        response = UMclient.account(recvWindow=6000)
        assets = response["assets"]
        positions = response["positions"]

        for asset_info in assets:
            if asset_info["asset"]=="USDT":
                asset_USDT = asset_info
        
        for position_info in positions:
            if position_info["symbol"]=="BTCUSDT":
                position_BTCUSDT = position_info

        return asset_USDT, position_BTCUSDT

    except ClientError as error:
        "Found error. status: {}, error code: {}, error message: {}".format(
            error.status_code, error.error_code, error.error_message
        )

In [3]:
a, p = get_account(um_futures_client)

In [2]:
serverTime = um_futures_client.time()['serverTime']
try:
    response = um_futures_client.get_position_mode(serverTime=serverTime)
    print(response)  # dualSidePosition = False, 즉 One-way Mode이고 Hedge 모드 아님. 
except ClientError as error:
    logging.error(
        "Found error. status: {}, error code: {}, error message: {}".format(
            error.status_code, error.error_code, error.error_message
        )
    )

{'dualSidePosition': False}


### One-Way Mode VS Hedge Mode
One-way 모드는 하나의 contract에 하나의 position만 잡을 수 있다.  
Hedge 모드는 하나의 contract에 두 개 이상의 Position을 잡을 수 있음.  
두 개 이상의 position을 잡을 수 있다는 게 무슨 말...

In [2]:
listen_key = um_futures_client.new_listen_key()["listenKey"]

In [3]:
# defining websocket streams
SYMBOL = "btcusdt"

## params of each streams
p_aggTrade = dict()
p_markPrice = dict(
    speed=1,
)

## make each streams
aggTrade_stream = f"{SYMBOL}@aggTrade"
markPrice_stream = f"{SYMBOL}@markPrice@{p_markPrice['speed']}s"
listen_key = listen_key

## make stream lists
stream = [
    aggTrade_stream,
    # markPrice_stream,
    listen_key,
]
# END


config_logging(logging, logging.DEBUG)

def message_handler(message):
    print(message, "\ntype : ", type(message))

my_client.start()


gd = GetData()

my_client.live_subscribe(
    stream=stream,
    id=1,
    callback=message_handler,
)

time.sleep(30)

logging.debug("closing ws connection")
my_client.stop()

{'id': 1, 'result': None} 
type :  <class 'dict'>
{'stream': 'btcusdt@aggTrade', 'data': {'e': 'aggTrade', 'E': 1674921418500, 'a': 140972530, 's': 'BTCUSDT', 'p': '23025.70', 'q': '19.219', 'f': 248987003, 'l': 248987003, 'T': 1674921418348, 'm': False}} 
type :  <class 'dict'>
{'stream': '8Aybe2f5iWJzpCU0i2uuNamPnMXknQcEY8GjBlFTbvEMQvbhTzWmNRzUlB5y77H5', 'data': {'e': 'ORDER_TRADE_UPDATE', 'T': 1674921418659, 'E': 1674921418661, 'o': {'s': 'BTCUSDT', 'c': 'web_sAbJcmGW4znFozsfH80T', 'S': 'BUY', 'o': 'MARKET', 'f': 'GTC', 'q': '0.100', 'p': '0', 'ap': '0', 'sp': '0', 'x': 'NEW', 'X': 'NEW', 'i': 3278967386, 'l': '0', 'z': '0', 'L': '0', 'n': '0', 'N': 'USDT', 'T': 1674921418659, 't': 0, 'b': '0', 'a': '0', 'm': False, 'R': False, 'wt': 'CONTRACT_PRICE', 'ot': 'MARKET', 'ps': 'BOTH', 'cp': False, 'rp': '0', 'pP': False, 'si': 0, 'ss': 0}}} 
type :  <class 'dict'>
{'stream': '8Aybe2f5iWJzpCU0i2uuNamPnMXknQcEY8GjBlFTbvEMQvbhTzWmNRzUlB5y77H5', 'data': {'e': 'ACCOUNT_UPDATE', 'T': 16749214

In [20]:
## 새 오더 작성하기
# 0. 현재 가격 일단 알아와보자
from datetime import datetime
def ms2dt(ms):
    return datetime.fromtimestamp(ms//1000)

from copy import deepcopy

dt_markPrice = data_markPrice.copy()
dt_aggTrade = data_aggTrade.copy()

In [21]:
dt_markPrice['E'] = dt_markPrice.E.apply(ms2dt)
dt_aggTrade['E'] = dt_aggTrade.E.apply(ms2dt)

In [54]:
import numpy as np
from cert.myfuncs import *
price_list = dt_markPrice['p'].astype(float)[len(dt_markPrice)-10:]
mean_price = price_list.mean()
last_price = price_list.tolist()[-1]
std_price = price_list.std()
volatility = realized_volatility(log_return(price_list))

0.00011046708018171482

In [49]:
std_price

1.046151242218611

In [ ]:
# 1. Buy(Long) Limit order

In [ ]:
# defining websocket streams
SYMBOL = "btcusdt"

## params of each streams
p_aggTrade = dict()
p_markPrice = dict(
    speed=1,
)

## make each streams
aggTrade_stream = f"{SYMBOL}@aggTrade"
markPrice_stream = f"{SYMBOL}@markPrice@{p_markPrice['speed']}s"

## make stream lists
stream = [
    aggTrade_stream,
    markPrice_stream,
]
# END




In [2]:
# defining websocket streams
SYMBOL = "btcusdt"

## params of each streams
p_aggTrade = dict()
p_markPrice = dict(
    speed=1,
)

## make each streams
aggTrade_stream = f"{SYMBOL}@aggTrade"
markPrice_stream = f"{SYMBOL}@markPrice@{p_markPrice['speed']}s"

## make stream lists
stream = [
    aggTrade_stream,
    markPrice_stream,
]
# END


# futures REST API
config_logging(logging, logging.DEBUG)

## make order : should be moved to trade_rules.
try:
    response = um_futures_client.new_order_test(
        symbol="BTCUSDT",
        side="SELL",
        type="MARKET",
        quantity=0.01,
    )
    logging.info(response)
except ClientError as error:
    logging.error(
        "Found error. status: {}, error code: {}, error message: {}".format(
            error.status_code, error.error_code, error.error_message
        )
    )
## END

DEBUG:root:url: https://testnet.binancefuture.com/fapi/v1/order/test
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): testnet.binancefuture.com:443
DEBUG:urllib3.connectionpool:https://testnet.binancefuture.com:443 "POST /fapi/v1/order/test?symbol=BTCUSDT&side=SELL&type=MARKET&quantity=0.01&timestamp=1674919221482&signature=3f4b8c02bef823e814eea058c45640075e47257bfe0be673727ccbdf846ae624 HTTP/1.1" 200 None
DEBUG:root:raw response from server:{"orderId":null,"symbol":"null","status":"null","clientOrderId":"null","price":"null","avgPrice":"null","origQty":"null","executedQty":"null","cumQuote":"null","timeInForce":"null","type":"null","reduceOnly":null,"closePosition":null,"side":"null","positionSide":"null","stopPrice":"null","workingType":"null","priceProtect":null,"origType":"null","updateTime":null}
INFO:root:{'orderId': None, 'symbol': 'null', 'status': 'null', 'clientOrderId': 'null', 'price': 'null', 'avgPrice': 'null', 'origQty': 'null', 'executedQty': 'null', 'cum

In [7]:
## check orders
try:
    response = um_futures_client.get_orders(symbol="BTCUSDT", recvWindow=2000)
    logging.info(response)
except ClientError as error:
    logging.error(
        "Found error. status: {}, error code: {}, error message: {}".format(
            error.status_code, error.error_code, error.error_message
        )
    )
## END

DEBUG:root:url: https://testnet.binancefuture.com/fapi/v1/openOrders
DEBUG:urllib3.connectionpool:https://testnet.binancefuture.com:443 "GET /fapi/v1/openOrders?symbol=BTCUSDT&recvWindow=2000&timestamp=1674460653712&signature=a6535662e0edd3b04d862e8ff55ed3156c0113b505b13c9e6e9b3cfbc00392f4 HTTP/1.1" 200 2
DEBUG:root:raw response from server:[]
INFO:root:[]


In [3]:
# defining websocket streams
SYMBOL = "btcusdt"

## params of each streams
p_aggTrade = dict()
p_markPrice = dict(
    speed=1,
)

## make each streams
aggTrade_stream = f"{SYMBOL}@aggTrade"
markPrice_stream = f"{SYMBOL}@markPrice@{p_markPrice['speed']}s"

## make stream lists
stream = [
    aggTrade_stream,
    markPrice_stream,
]
# END

config_logging(logging, logging.DEBUG)

def message_handler(message):
    print(message, "\ntype : ", type(message))

my_client.start()



gd = GetData()

my_client.live_subscribe(
    stream=stream,
    id=1,
    callback=gd.event_2_DataFrame,
)

time.sleep(30)

logging.debug("closing ws connection")
my_client.stop()

INFO:root:Connection with URL: wss://stream.binancefuture.com/stream
INFO:root:Start to connect....
INFO:root:Server connected
INFO:root:Sending message to Server: b'{"method": "SUBSCRIBE", "params": ["btcusdt@aggTrade", "btcusdt@markPrice@1s"], "id": 1}'


{'id': 1, 'result': None} 
type :  <class 'dict'>


DEBUG:root:closing ws connection


In [6]:
data_markPrice = gd.df_dict['markPriceUpdate']
data_aggTrade = gd.df_dict['aggTrade']

In [5]:
gd.df_dict['aggTrade']

,e,E,a,s,p,q,f,l,T,m
0,aggTrade,1674919235442,140971395,BTCUSDT,23032.20,0.001,248985826,248985826,1674919235289,False
1,aggTrade,1674919235942,140971396,BTCUSDT,23032.20,0.055,248985827,248985827,1674919235785,False
2,aggTrade,1674919237947,140971397,BTCUSDT,23030.10,0.001,248985828,248985828,1674919237943,True
3,aggTrade,1674919237948,140971398,BTCUSDT,23025.10,0.001,248985829,248985829,1674919237943,True
4,aggTrade,1674919238099,140971399,BTCUSDT,23022.80,0.967,248985830,248985830,1674919237943,True
5,aggTrade,1674919240148,140971400,BTCUSDT,23032.20,3.879,248985831,248985831,1674919239996,False
6,aggTrade,1674919243648,140971401,BTCUSDT,23026.10,0.001,248985832,248985832,1674919243644,False
7,aggTrade,1674919243650,140971402,BTCUSDT,23031.10,0.001,248985833,248985833,1674919243644,False
8,aggTrade,1674919243801,140971403,BTCUSDT,23032.20,0.998,248985834,248985834,1674919243645,False
9,aggTrade,1674919252237,140971404,BTCUSDT,23032.20,2.619,248985835,248985835,1674919252084,False


In [7]:
import logging
from binance.um_futures import UMFutures
from binance.lib.utils import config_logging
from binance.error import ClientError

config_logging(logging, logging.DEBUG)

from cert import binanceKey
from cert import test_binanceKey

key = binanceKey.API_KEY
secret = binanceKey.SECRET_KEY

um_futures_client = UMFutures(key=key, secret=secret, base_url)

try:
    response = um_futures_client.new_order_test(
        symbol="BTCUSDT",
        side="SELL",
        type="MARKET",
        quantity=0.001,
    )
    logging.info(response)
except ClientError as error:
    logging.error(
        "Found error. status: {}, error code: {}, error message: {}".format(
            error.status_code, error.error_code, error.error_message
        )
    )

DEBUG:root:url: https://fapi.binance.com/fapi/v1/order/test
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): fapi.binance.com:443
DEBUG:urllib3.connectionpool:https://fapi.binance.com:443 "POST /fapi/v1/order/test?symbol=BTCUSDT&side=SELL&type=MARKET&quantity=0.001&timestamp=1674457717892&signature=bd4bf488a33c40b01996aea1e240d1e0a8369c3668d43e460d372f49bfcf66f9 HTTP/1.1" 200 None
DEBUG:root:raw response from server:{"orderId":null,"symbol":"null","status":"null","clientOrderId":"null","price":"null","avgPrice":"null","origQty":"null","executedQty":"null","cumQuote":"null","timeInForce":"null","type":"null","reduceOnly":null,"closePosition":null,"side":"null","positionSide":"null","stopPrice":"null","workingType":"null","priceProtect":null,"origType":"null","updateTime":null}
INFO:root:{'orderId': None, 'symbol': 'null', 'status': 'null', 'clientOrderId': 'null', 'price': 'null', 'avgPrice': 'null', 'origQty': 'null', 'executedQty': 'null', 'cumQuote': 'null', 'timeInFor

In [1]:
import logging
from binance.um_futures import UMFutures
from binance.lib.utils import config_logging
from binance.error import ClientError

config_logging(logging, logging.DEBUG)

from cert import test_binanceKey

key = test_binanceKey.API_KEY
secret = test_binanceKey.SECRET_KEY
futures_testnet = test_binanceKey.futures_testnet
futures_websocket_testnet = test_binanceKey.futures_websocket_testnet

um_futures_client = UMFutures(key=key, secret=secret, base_url=futures_testnet)

try:
    response = um_futures_client.get_orders(symbol="BTCUSDT", recvWindow=2000)
    logging.info(response)
except ClientError as error:
    logging.error(
        "Found error. status: {}, error code: {}, error message: {}".format(
            error.status_code, error.error_code, error.error_message
        )
    )

DEBUG:root:url: https://testnet.binancefuture.com/fapi/v1/openOrders
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): testnet.binancefuture.com:443
DEBUG:urllib3.connectionpool:https://testnet.binancefuture.com:443 "GET /fapi/v1/openOrders?symbol=BTCUSDT&recvWindow=2000&timestamp=1674459396769&signature=b60a7a0d8fc327d2898260045b227f300c562c75fec235c3ed557713e70f2c56 HTTP/1.1" 200 2
DEBUG:root:raw response from server:[]
INFO:root:[]
